In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ast
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import normalize
from keras.preprocessing import image
from time import time
from scipy.spatial.distance import cosine, euclidean, hamming

2024-10-11 19:00:08.550834: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-11 19:00:08.727613: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-11 19:00:08.976819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 19:00:09.311727: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 19:00:09.312966: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 19:00:09.653198: I tensorflow/core/platform/cpu_feature_guard.cc:

#### dataset linl:https://www.kaggle.com/datasets/elisaxxygao/foodrecsysv1/data

In [3]:
recipe = pd.read_csv('archive/raw-data_recipe.csv')
recipe.head()

,recipe_id,recipe_name,aver_rate,image_url,review_nums,ingredients,cooking_directions,nutritions,reviews
0,222388,Homemade Bacon,5.000000,https://images.media-allrecipes.com/userphotos...,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{u'niacin': {u'hasCompleteData': False, u'name...","{8542392: {'rating': 5, 'followersCount': 11, ..."
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,https://images.media-allrecipes.com/userphotos...,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name...","{3574785: {'rating': 5, 'followersCount': 0, '..."
2,218939,Foolproof Rosemary Chicken Wings,4.571429,https://images.media-allrecipes.com/userphotos...,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'...","{13774946: {'rating': 5, 'followersCount': 0, ..."
3,87211,Chicken Pesto Paninis,4.625000,https://images.media-allrecipes.com/userphotos...,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1563136: {'rating': 5, 'followersCount': 0, '..."
4,245714,Potato Bacon Pizza,4.500000,https://images.media-allrecipes.com/userphotos...,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{2945555: {'rating': 5, 'followersCount': 6690..."


#### Directions of recipe based on recipe_id

In [20]:
import ast

# Assuming `recipe` is a DataFrame with a 'recipe_id' and 'cooking_directions' column
# Filter the DataFrame for the specific recipe_id and get the 'cooking_directions' string
recipe_str = recipe[recipe['recipe_id'] == 218939]['cooking_directions'].iloc[0]

# Use ast.literal_eval to convert the string into a dictionary
recipe_dict = ast.literal_eval(recipe_str)

# Access the 'directions' value
directions = recipe_dict['directions']

# Print the directions
print(directions)




Prep
20 m
Cook
40 m
Ready In
1 h
Preheat an oven to 350 degrees F (175 degrees C).
Arrange the chicken wings, rosemary, and garlic cloves on a broiler-proof baking sheet, making sure the chicken pieces are not touching. Drizzle the olive oil over the chicken and garlic. Season the wings on all sides with the lemon pepper and seasoned salt.
Bake in the preheated oven until chicken meat is no longer pink at the bone and the juices run clear, 35 to 40 minutes, turning the wings once about halfway through the cooking time. An instant-read thermometer inserted into the thickest part of a wing should read at least 160 degrees F (70 degrees C).
Take the baking sheet from the oven and set the oven's broiler to High. Remove the garlic and rosemary from the sheet and set aside. Turn the wings again.
Broil the wings until golden brown, about 5 minutes. Garnish with the rosemary sprigs and garlic to serve.


In [22]:
import ast

# Define a function to extract and print cooking directions based on recipe_id
def get_cooking_directions(recipe_df, recipe_id):
    try:
        # Filter the DataFrame to find the row corresponding to the given recipe_id
        recipe_str = recipe_df[recipe_df['recipe_id'] == recipe_id]['cooking_directions'].iloc[0]
        recipe_name=recipe_df[recipe_df['recipe_id'] == recipe_id]['recipe_name'].iloc[0]
        # Use ast.literal_eval to convert the string into a dictionary
        recipe_dict = ast.literal_eval(recipe_str)

        # Access the 'directions' value from the dictionary
        directions = recipe_dict.get('directions', 'Directions not found')

        # Print the cooking directions
        print(f"Cooking Directions for {recipe_name}:\n{directions}")

    except IndexError:
        print(f"Error: Recipe ID {recipe_id} not found.")
    except ValueError:
        print("Error: Unable to parse the recipe directions.")

# Example usage:
# Assuming `recipe_df` is a DataFrame that contains 'recipe_id' and 'cooking_directions' columns
# Call the function with a DataFrame and recipe_id
get_cooking_directions(recipe, 222388)


Cooking Directions for Homemade Bacon:
Prep
5 m
Cook
2 h 45 m
Ready In
11 h 50 m
Preheat oven to 200 degrees F (95 degrees C).
Season pork belly with paprika, salt, and pepper. Tightly wrap pork twice in heavy-duty aluminum foil. Place on a baking sheet and bake in the preheated oven for 2 1/2 hours. Turn off the oven; let pork rest in the oven for 1 hour. Remove meat from oven, leaving it wrapped in aluminum foil, and refrigerate at least 8 hours or overnight.
Remove pork from foil and slice across the grain in 1/4-inch thick slices. Working in batches, cook pork in a non-stick skillet over medium heat until golden and crisped, 6 to 8 minutes per slice.


In [ ]:
recipe.shape

In [ ]:
recipe.isna().sum()

In [ ]:
# recipe = recipe.drop(columns=['cooking_directions', 'reviews'])

In [ ]:
recipe.shape

In [ ]:
# round average rating into 2 decimal places
def avg_rate(col):
    return f'{col:.2f}'
recipe.aver_rate = recipe.aver_rate.apply(avg_rate)
recipe.head()

In [ ]:
recipe.aver_rate = recipe.aver_rate.astype(float)
recipe.dtypes
list_of_dict = []

for row in recipe.nutritions:
    list_of_dict.append(ast.literal_eval(row))
list_of_dict[1]


# extract percent daily values for selected nutritions
calories_list = []
fat_list = []
carbohydrates_list = []
protein_list = []
cholesterol_list = []
sodium_list = []
fiber_list = []

for x in range(len(list_of_dict)):
    calories_list.append(list_of_dict[x]['calories']['percentDailyValue'])
    fat_list.append(list_of_dict[x]['fat']['percentDailyValue'])
    carbohydrates_list.append(list_of_dict[x]['carbohydrates']['percentDailyValue'])
    protein_list.append(list_of_dict[x]['protein']['percentDailyValue'])
    cholesterol_list.append(list_of_dict[x]['cholesterol']['percentDailyValue'])
    sodium_list.append(list_of_dict[x]['sodium']['percentDailyValue'])
    fiber_list.append(list_of_dict[x]['fiber']['percentDailyValue'])
    
    
# group all the data into dataframe
data = {'calories': calories_list, 'fat': fat_list, 'carbohydrates': carbohydrates_list, 
       'protein': protein_list, 'cholesterol': cholesterol_list, 'sodium': sodium_list, 
       'fiber': fiber_list}

df = pd.DataFrame(data)
df.index = recipe['recipe_id']
df.head()

In [ ]:
print(df.shape)
df = df.dropna()
def text_cleaning(cols):
    if cols == '< 1':
        return 1
    else:
        return cols

for col in df.columns:
    df[col] = df[col].apply(text_cleaning)
df = df.apply(pd.to_numeric)
# normalized nutrition data by columns
df_normalized = pd.DataFrame(normalize(df, axis=0))
df_normalized.columns = df.columns
df_normalized.index = df.index
df_normalized.head()


# show recipe id, recipe name and image of selected recipe
def selected_recipe(recipe_id):
    image_path = "archive/raw-data-images/raw-data-images/{}.jpg"
    image_path = image_path.format(recipe_id)
    
    img = image.load_img(image_path)
    img = image.img_to_array(img, dtype='int')
        
    fig, ax = plt.subplots(1,1)
    ax.imshow(img)
    ax.axis('off')
    
    recipe_df = recipe.set_index('recipe_id')
    x = "{}  {}".format(recipe_id, recipe_df.at[recipe_id, 'recipe_name'])
    ax.set_title(x)
    
    print(df.loc[df.index.isin([recipe_id])].transpose())
    
selected_recipe(240488)


recipe_final= pd.merge(recipe, df, on='recipe_id')
recipe_final= recipe_final.drop(columns=['nutritions'])

recipe_final.to_csv("recipe_final_full.csv")

In [ ]:
ingredients_list = []

for x in range(len(recipe_final)):
    ingredients_list.append(recipe_final['ingredients'][x].split('^'))
data = {'ingredients_list': ingredients_list}
df1 = pd.DataFrame(data)
df1.index = recipe_final['recipe_id']
df1.head()

recipe_final= pd.merge(recipe_final, df1, on='recipe_id')
recipe_final= recipe_final.drop(columns=['ingredients'])
recipe_final.head()

recipe_final.to_csv("recipe_final.csv")


In [ ]:
final_df1=pd.read_csv('recipe_final_full.csv')
final_df1

In [ ]:
#checking for cooking directions
get_cooking_directions(final_df1,222388)

In [ ]:
final_df2=pd.read_csv('recipe_final.csv')
final_df2